### 1 часть

Решить нелинейное уравнение
f(x) = 0
f — любая гладкая, с заранее известным нам корнем
1) методом деления пополам
2) методом Ньютона (с диагностикой кратности корня)

На доске пример такого уравнения

### 2 часть

Решить нелинейную систему 3 на 3

In [1]:
!. ./.venv/bin/activate
import numpy as np

## 1. Метод деления пополам
Уравнение
$$
e^{x} = 3.5
$$
Точное решение: $x = \ln(3.5) \approx 1.253$

In [2]:
def dichotomy_iter(l0, r0, eps, max_iter, f):
  l, r = l0, r0
  prev_mid = 0
  for i in range(max_iter):
    mid = l + (r - l) / 2
    if (f(mid) * f(l) < 0) :
      r = mid
    elif (f(mid) * f(r) < 0):
      l = mid
  
    if (abs(prev_mid - mid) < eps):
      return mid
    prev_mid = mid
    

def f(x):
  return np.exp(x) - 3.5

EPS = 1e-3
MAX_ITER = 100
print(f'Решение {dichotomy_iter(0, 2, EPS, MAX_ITER, f)}')

Решение 1.2529296875


## 2. Метод Ньютона
Уравнение
$$
(e^{x} - 3.5)^2 = 0
$$
Точное решение: $x = \ln(3.5) \approx 1.253$

In [30]:
import sympy as sp

def df(f, x0, eps=1e-6):
   return ( f(x0 + eps) - f(x0) ) / eps

def newton_iter(x0, f, max_iter, eps):
  x_cur = x0
  for i in range(max_iter):
    try:
      x_new = x_cur - f(x_cur)/df(f, x_cur)
    except ZeroDivisionError:
      print('Zero division encountered')
      return None
    
    if abs(x_new - x_cur) < eps:
      return x_new
    x_cur = x_new

  return x_new

def f(x):
  return np.exp(x) - 3.5

def root_multiplicity(df, root, eps=1e-10, max_order=10):    
    x = sp.symbols('x')
    derivatives = [ df ]  
    for _ in range(max_order):
      derivatives.append( sp.diff(derivatives[-1], x) )
    
    derivatives = [ sp.lambdify(x, derivatives[i]) for i in range(len(derivatives)) ]

    # Проверка кратности корня
    for m in range(1, len(derivatives) + 1):
        f_m = derivatives[m-1]
        if abs(f_m(root)) > eps:
            return f"Корень имеет кратность {m}"
    
    return f"Корень имеет кратность больше {len(derivatives)}"

EPS = 1e-3
MAX_ITER = 1000
root = newton_iter(3, f, MAX_ITER, EPS)
print(f'Решение: {root}')

x = sp.symbols('x')
f_ = (sp.exp(x) - 3.5)**2
df_ = sp.diff(f_, x)
f_num = sp.lambdify(x, f_)
df_num = sp.lambdify(x, df_)

result = root_multiplicity(df_, root)
print(result)

Решение: 1.2527629684957866
Корень имеет кратность 2


## 3. Решение нелинейной системы 3x3

$$
\begin{cases}
  x = 1 + \cos{y} \\
  y = 0.3 + \sin{z} \\
  z = \frac{x}{6}
\end{cases}
$$
$ x \approx 1.82557; y \approx 0.599589; z \approx 0.304261 $

In [4]:
def f(x0):
  x, y, z = x0[0], x0[1], x0[2]
  f1 = np.cos(y) + 1 - x
  f2 = 0.3 + np.sin(z) - y
  f3 = x/6 - z

  return [f1, f2, f3]

def J(x0):
  J = np.ones([3,3])
  x, y, z = x0[0], x0[1], x0[2]
  J[0,0] = -1
  J[0,1] = -np.sin(y)
  J[0,2] = 0

  J[1,0] = 0
  J[1,1] = -1
  J[1,2] = np.cos(z)

  J[2,0] = 1/6
  J[2,1] = 0
  J[2,2] = -1

  return J

def newton_sys(x0, f, eps, max_iter):
  x_cur = x0

  for i in range(max_iter):
    dx = -np.linalg.inv(J(x_cur))@f(x_cur)
    x_new = x_cur + dx

    if np.linalg.norm(x_new - x_cur) < eps:
      return x_new
    x_cur = x_new

  return x_cur

EPS=1e-3
MAX_ITER=100
print(f'Решение: {newton_sys([1,1,1], f, EPS, MAX_ITER)}')

Решение: [1.82556789 0.5995885  0.30426132]
